In [1]:
import pandas as pd
import requests
import urllib.request as req
import json
from bs4 import BeautifulSoup
import requests.packages.urllib3
import json
import re

requests.packages.urllib3.disable_warnings() #使用 requests 時發生 InsecureRequestWarning: Unverified HTTPS request is being made

# **爬文**

In [2]:
def contenter(url):
    request = req.Request(url, headers = {
    "cookie": "over18=1",    
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    soup = BeautifulSoup(data, "html.parser")

    main_container = soup.find(id = 'main-container')
    all_text = main_container.text
    pre_texts = all_text.split('※ 發信站')[:-1]
    # 將前面的所有內容合併成一個
    one_text = "--".join(pre_texts)
    one_text
    #url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+')
    #new_string = url_pattern.sub(r'', new_string)
    #new_string
    # 以\n切割，第一行標題不要
    texts = one_text.split("\n")[1:]
    li_text = []
    for i in texts:
        if ('※' not in i) & (len(i) != 0):
            li_text.append(i)
    content = " ".join(li_text)
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+')
    content = url_pattern.sub(r'', content)
    return content

In [42]:
import time
title = []
date = []
content = []
for page in range(1, 4): #543
    url = 'https://www.ptt.cc/bbs/Gossiping/search?page=' + str(page) +'&q=%E9%99%B3%E6%99%82%E4%B8%AD'#q=候選人名
    request = req.Request(url, headers = {
        "cookie": "over18=1",    
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    
    soup = BeautifulSoup(data, "html.parser")
    titles = soup.find_all('div', class_= 'r-ent')
    dt = soup.find_all('div', class_ = 'date')
    t_d = dict(zip(titles, dt))


    try:
        for t, d in t_d.items():
            if t.a != None:
                title.append(t.a.string)
                date.append(d.string)
                url = "https://www.ptt.cc"+t.a.get("href")
                cont = contenter(url)
                if len(cont) != 0:
                    content.append(cont)
                else:
                    content.append(None)
            else:
                pass
    except:
        print('第{}頁'.format(page))
    time.sleep(5)
    

dic = {
    '標題': title,
    '發布日期': date,
    '內文': content
}


ptt_test = pd.DataFrame(dic)

In [43]:
ptt_test

,標題,發布日期,內文
0,[新聞] 找狗仔跟拍陳時中？徐巧芯截圖提告 檢,7/05,作者TravelFar ()看板Gossiping標題[新聞] 找狗仔跟拍陳時中？徐巧芯截圖...
1,[新聞] 蔡英文親邀賴清德團聚3小時 陳時中：小英,6/25,作者LIN9 (LIN9)看板Gossiping標題[新聞] 蔡英文親邀賴清德團聚3小時 陳...
2,[新聞] 餵藥案延燒 陳時中質疑侯友宜：醫療小組,6/20,作者yinnie (Alexander)看板Gossiping標題[新聞] 餵藥案延燒 陳時...
3,[新聞]陳時中自嘲「失敗組」：團結勝利組支持,6/20,作者yushenglu (子欲養而親不待)看板Gossiping標題[新聞]陳時中自嘲「失敗...
4,[新聞] 快訊／新北餵藥案全國普篩？ 陳時中反,6/19,作者zoosleep (王者)看板Gossiping標題[新聞] 快訊／新北餵藥案全國普篩？...
5,[新聞] 質疑新北市府效率 陳時中：拖18天還強,6/19,作者zzyyxx77 (討噓大師77)看板Gossiping標題[新聞] 質疑新北市府效率 ...
6,[新聞]幼兒園餵藥案 陳時中質疑新北市府行政效,6/19,作者yushenglu (子欲養而親不待)看板Gossiping標題[新聞]幼兒園餵藥案 陳...
7,Re: [新聞] 新北餵藥案 陳時中：拖18天還強調第一時,6/18,作者hamsters56 (鼠鼠so cute)看板Gossiping標題Re: [新聞] ...
8,[新聞] 新北餵藥案 陳時中：拖18天還強調第一時,6/18,作者gasara (賭盤開了 收錢 ^O^/)看板Gossiping標題[新聞] 新北餵藥案...
9,Re: [新聞] 餵藥案延燒 陳時中質疑侯友宜：醫療小組,6/18,作者ukalm (我回來了.....爆出來追查)看板Gossiping標題Re: [新聞] ...


# **整理文章**

In [47]:
ptt_test.drop_duplicates(subset=['發布日期'], keep='first', inplace=True)
dt = dict(zip(ptt_test['發布日期'], ptt_test['內文']))
review = []
for k, v in dt.items():
    clean_content = v.split('2023')[1]#篩掉作者標題時間 保留文章
    review.append(clean_content)

In [48]:
ptt_test['內文(clean)'] = review
ptt_test

,標題,發布日期,內文,內文(clean)
0,[新聞] 找狗仔跟拍陳時中？徐巧芯截圖提告 檢,7/05,作者TravelFar ()看板Gossiping標題[新聞] 找狗仔跟拍陳時中？徐巧芯截圖...,1.媒體來源: NOWnews 2.記者署名: 記者潘千詩／台北報導 3.完整新聞標題: ...
1,[新聞] 蔡英文親邀賴清德團聚3小時 陳時中：小英,6/25,作者LIN9 (LIN9)看板Gossiping標題[新聞] 蔡英文親邀賴清德團聚3小時 陳...,1.媒體來源:自由 2.記者署名:陳政宇 3.完整新聞標題:蔡英文親邀賴清德團聚3小時 陳...
2,[新聞] 餵藥案延燒 陳時中質疑侯友宜：醫療小組,6/20,作者yinnie (Alexander)看板Gossiping標題[新聞] 餵藥案延燒 陳時...,餵藥案延燒 陳時中質疑侯友宜：醫療小組拖18天還說第一時間組成？
4,[新聞] 快訊／新北餵藥案全國普篩？ 陳時中反,6/19,作者zoosleep (王者)看板Gossiping標題[新聞] 快訊／新北餵藥案全國普篩？...,備註請放最後面 違者新聞文章刪除 1.媒體來源: ettoday 2.記者署名: 呂晏慈 ...
7,Re: [新聞] 新北餵藥案 陳時中：拖18天還強調第一時,6/18,作者hamsters56 (鼠鼠so cute)看板Gossiping標題Re: [新聞] ...,: 1.媒體來源: : 自由時報 : 2.記者署名: : 陳昀 : 3.完整新聞標題: :...
13,[新聞]陳時中快閃金門「催血」 籲全民挽袖捐血,6/07,作者yushenglu (子欲養而親不待)看板Gossiping標題[新聞]陳時中快閃金門「...,陳時中快閃金門「催血」 籲全民挽袖捐血做公益 20:38
14,[新聞] 陳時中快閃金門出席捐血活動 小英之友會,6/06,作者lamigo (lamigo)看板Gossiping標題[新聞] 陳時中快閃金門出席捐血...,1.媒體來源:聯合 2.記者署名:蔡家蓁 3.完整新聞標題:陳時中快閃金門出席捐血活動 小...
16,Re: [新聞] 小英之友會也傳性騷擾 陳時中:會瞭解,6/04,作者jeff17 (step up)看板Gossiping標題Re: [新聞] 小英之友會也...,"根據TVBS的報導,這個小英之友會的重要人士, 跟陳時中選台北市長後援會總會長是同一個人 ..."
17,[新聞] 對性騷擾零容忍 陳時中：絕不姑息養奸,6/03,作者takechance (dfs)看板Gossiping標題[新聞] 對性騷擾零容忍 陳時...,民進黨內性騷事件連環爆，前衛福部長陳時中今表示，台灣在性別平等上還不算是成熟社 會，除未...
25,[新聞] 涉性騷導演現身陳時中活動 徐巧芯：陸委,6/01,作者rguqong (Algernon)看板Gossiping標題[新聞] 涉性騷導演現身陳...,涉性騷導演現身陳時中活動 徐巧芯：陸委會是否給他48.5萬標案？


# **分成新聞內文和鄉民評論**

In [36]:
df_news = ptt_test[~ptt_test['標題'].str.contains('Re:')]
df_review = ptt_test[ptt_test['標題'].str.contains('Re:')]

In [40]:
df_news
#df_review

,標題,發布日期,內文,內文(clean)
0,[新聞] 找狗仔跟拍陳時中？徐巧芯截圖提告 檢,7/05,作者TravelFar ()看板Gossiping標題[新聞] 找狗仔跟拍陳時中？徐巧芯截圖...,1.媒體來源: NOWnews 2.記者署名: 記者潘千詩／台北報導 3.完整新聞標題: ...
1,[新聞] 蔡英文親邀賴清德團聚3小時 陳時中：小英,6/25,作者LIN9 (LIN9)看板Gossiping標題[新聞] 蔡英文親邀賴清德團聚3小時 陳...,1.媒體來源:自由 2.記者署名:陳政宇 3.完整新聞標題:蔡英文親邀賴清德團聚3小時 陳...
2,[新聞] 餵藥案延燒 陳時中質疑侯友宜：醫療小組,6/20,作者yinnie (Alexander)看板Gossiping標題[新聞] 餵藥案延燒 陳時...,餵藥案延燒 陳時中質疑侯友宜：醫療小組拖18天還說第一時間組成？
4,[新聞] 快訊／新北餵藥案全國普篩？ 陳時中反,6/19,作者zoosleep (王者)看板Gossiping標題[新聞] 快訊／新北餵藥案全國普篩？...,備註請放最後面 違者新聞文章刪除 1.媒體來源: ettoday 2.記者署名: 呂晏慈 ...
13,[新聞]陳時中快閃金門「催血」 籲全民挽袖捐血,6/07,作者yushenglu (子欲養而親不待)看板Gossiping標題[新聞]陳時中快閃金門「...,陳時中快閃金門「催血」 籲全民挽袖捐血做公益 20:38
14,[新聞] 陳時中快閃金門出席捐血活動 小英之友會,6/06,作者lamigo (lamigo)看板Gossiping標題[新聞] 陳時中快閃金門出席捐血...,1.媒體來源:聯合 2.記者署名:蔡家蓁 3.完整新聞標題:陳時中快閃金門出席捐血活動 小...
17,[新聞] 對性騷擾零容忍 陳時中：絕不姑息養奸,6/03,作者takechance (dfs)看板Gossiping標題[新聞] 對性騷擾零容忍 陳時...,民進黨內性騷事件連環爆，前衛福部長陳時中今表示，台灣在性別平等上還不算是成熟社 會，除未...
25,[新聞] 涉性騷導演現身陳時中活動 徐巧芯：陸委,6/01,作者rguqong (Algernon)看板Gossiping標題[新聞] 涉性騷導演現身陳...,涉性騷導演現身陳時中活動 徐巧芯：陸委會是否給他48.5萬標案？
27,[新聞]才傳徵召鄭家純 陳時中表態：盼吳沛憶成功,5/31,作者yushenglu (奶茶古拳法)看板Gossiping標題[新聞] 時間Wed May...,才傳徵召鄭家純 陳時中表態：盼吳沛憶成功 記者 李作珩 張寓科 / 攝影 吳奎炎 劉昱駿 ...
36,[新聞] 綠找雞排妹李正皓引基層炸鍋 陳時中:還沒,5/27,作者elmoman (\同志友善帳號/)看板Gossiping標題[新聞] 綠找雞排妹李正皓...,綠找雞排妹、李正皓引基層炸鍋 陳時中：還沒定案 聯合報 張曼蘋 民進黨宣布與名嘴李正皓合作...
